In [1]:
import os
from tfx.utils.dsl_utils import csv_input
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.proto import example_gen_pb2
from tfx.components import StatisticsGen, SchemaGen, ExampleValidator, Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

context = InteractiveContext()

In [4]:
# load credit card fraud dataset if not exists in data folder
import urllib.request
if not os.path.isfile("data/creditcard.csv"):
    if not os.path.exists("data"): os.mkdir("data")
    urllib.request.urlretrieve("https://uplanet-archives.s3.amazonaws.com/creditcard.csv", "data/creditcard.csv")

In [5]:
# load transactions
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=3),
                 example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
             ]))

examples = csv_input(os.path.join('', 'data'))
example_gen = CsvExampleGen(input=examples, output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-21T16_51_35.272987-9jk3xmks/CsvExampleGen/examples/2, id: 4)]
        ))

In [6]:
# compute stats
compute_eval_stats = StatisticsGen(
      examples=example_gen.outputs['examples']
      )
context.run(compute_eval_stats)
context.show(compute_eval_stats.outputs['statistics'])

/root/miniconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [7]:
# Generate schema
infer_schema = SchemaGen(
    statistics=compute_eval_stats.outputs['statistics'])

context.run(infer_schema)

context.show(infer_schema.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
"'""Amount""'",FLOAT,required,single,-
"'""Class""'",INT,required,single,-
"'""Time""'",FLOAT,required,single,-
"'""V1""'",FLOAT,required,single,-
"'""V10""'",FLOAT,required,single,-
"'""V11""'",FLOAT,required,single,-
"'""V12""'",FLOAT,required,single,-
"'""V13""'",FLOAT,required,single,-
"'""V14""'",FLOAT,required,single,-


In [8]:
# validate data
validate_stats = ExampleValidator(
      statistics=compute_eval_stats.outputs['statistics'],
      schema=infer_schema.outputs['schema']
      )

context.run(validate_stats)

context.show(validate_stats.outputs['anomalies'])

In [9]:
# transform data
transform_eval = Transform(
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    module_file=os.path.abspath('creditcard_fraud_transform.py'))

context.run(transform_eval)

context.show(transform_eval.outputs)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-03-21T16_51_35.272987-9jk3xmks/Transform/transform_graph/6/.temp_path/tftransform_tmp/2d20ad7a7a694158b01fadc1c3a325ea/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-03-21T16_51_35.272987-9jk3xmks/Transform/transform_graph/6, id: 8)]
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-03-21T16_51_35.272987-9jk3xmks/Transform/transformed_examples/6, id: 9)]
)}

In [28]:
from tfx.components import Trainer
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.proto import trainer_pb2

# train model
trainer = Trainer(
    module_file=os.path.abspath('creditcard_fraud_train.py'),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform_eval.outputs['transformed_examples'],
    transform_graph=transform_eval.outputs['transform_graph'],
    schema=infer_schema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=10000),
    eval_args=trainer_pb2.EvalArgs(num_steps=5000))
context.run(trainer)

['/tmp/tfx-interactive-2020-03-21T16_51_35.272987-9jk3xmks/Transform/transformed_examples/6/train/*']
/tmp/tfx-interactive-2020-03-21T16_51_35.272987-9jk3xmks/Transform/transform_graph/6
/tmp/tfx-interactive-2020-03-21T16_51_35.272987-9jk3xmks/Trainer/model/23/serving_model_dir
10000
5000


ValueError: The `label_key` provided ('Class_xf') must be one of the `features` keys.